# Preprocesamiento del dataset

This celd make the processing of the dataset provide by NASA in https://wufs.wustl.edu/SpaceApps/data/space_apps_2024_seismic_detection.zip

This is mandatory for the project pipeline because it returns the files processed with the labels of the seismic events.

For runing, it is needed to stablish the follow variables: catalogo_filepath (path to the catalogs with the events)
eventos_directory (path to the directory where are alocated the csv of the events)
output_directory (path to the directory where the finals csv are saved)

In [ ]:
import pandas as pd
import os
import numpy as np

# Función para cargar el archivo del evento
def cargar_evento(evento_filepath):
    evento_df = pd.read_csv(evento_filepath)
    # Convertir 'time_abs' a formato datetime
    evento_df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'] = pd.to_datetime(evento_df['time_abs(%Y-%m-%dT%H:%M:%S.%f)'],
                                                                 format='%Y-%m-%dT%H:%M:%S.%f')
    return evento_df

# Función para encontrar archivos .csv en subcarpetas de manera recursiva
def encontrar_archivo_csv(filename, eventos_directory):
    for root, dirs, files in os.walk(eventos_directory):  # Recorre todas las carpetas y archivos
        for file in files:
            if file == f"{filename}.csv":
                return os.path.join(root, file)  # Retorna la ruta completa del archivo si lo encuentra
    return None

# Función para combinar el evento con el catálogo y agregar etiqueta binaria
def combinar_y_etiquetar_evento(catalogo_evento, eventos_directory):
    # Obtener el nombre del archivo de evento desde el catálogo
    filename = catalogo_evento['filename']

    # Buscar el archivo correspondiente en las subcarpetas
    evento_filepath = encontrar_archivo_csv(filename, eventos_directory)

    # Verificar si el archivo existe
    if evento_filepath is None:
        print(f"Archivo {filename}.csv no encontrado.")
        return None

    # Cargar el archivo de evento
    evento_df = cargar_evento(evento_filepath)

    # Obtener el valor de 'time_rel' del catálogo que marca el inicio del evento
    time_rel_inicio = catalogo_evento['time_rel(sec)']

    # Crear una nueva columna 'label' con 0 en todas las filas
    evento_df['label'] = 0

    # Asignar 1 en la fila donde 'time_rel' coincida con el valor del catálogo
    evento_df.loc[round(evento_df['time_rel(sec)']) == time_rel_inicio, 'label'] = 1

    return evento_df

# Función para crear el directorio si no existe
def crear_directorio_si_no_existe(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directorio '{directory}' creado.")

# Función principal para procesar los eventos desde el catálogo
def procesar_catalogo(catalogo_filepath, eventos_directory, output_directory):
    # Cargar el catálogo
    catalogo_df = pd.read_csv(catalogo_filepath)

    # Crear el directorio de salida si no existe
    crear_directorio_si_no_existe(output_directory)

    # Iterar sobre cada fila del catálogo
    for _, catalogo_evento in catalogo_df.iterrows():
        # Combinar y etiquetar
        evento_etiquetado = combinar_y_etiquetar_evento(catalogo_evento, eventos_directory)

        if evento_etiquetado is not None:
            # Obtener el nombre del archivo de evento
            filename = catalogo_evento['filename']
            # Guardar el archivo con etiquetas
            output_filepath = os.path.join(output_directory, f"{filename}.csv")
            evento_etiquetado.to_csv(output_filepath, index=False)
            print(f"Evento {filename}.csv procesado y guardado en {output_filepath}")

# Ruta al archivo de catálogo y directorio de eventos
catalogo_filepath = '' # space_apps_2024_seismic_detection/data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv
eventos_directory = '' # space_apps_2024_seismic_detection/data/lunar/training/data
output_directory = ''

# Ejecutar el procesamiento
procesar_catalogo(catalogo_filepath, eventos_directory, output_directory)

In [ ]:
# Corroborar que se hayan creado todas las etiquetas
bin_files = [f for f in os.listdir(output_directory) if f.endswith('.csv')]
for file in bin_files:
    df = pd.read_csv(os.path.join(output_directory, file))
    if sorted(df['label'].unique()) != [0, 1]:
        print(f'En el archivo {file} falta etiqueta')

# Entrenamiento del modelo

In this training celd, the only variable to set is 'carpeta_eventos' with the same path to the directory where are the csv processed

In [ ]:
import os
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from keras.callbacks import ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import resample
from datetime import datetime
import matplotlib.pyplot as plt

def crear_directorio_si_no_existe(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directorio '{directory}' creado.")

# Función para crear el modelo LSTM
def crear_modelo_lstm():
    model = Sequential()
    model.add(Input(shape=(None, 1)))
    model.add(LSTM(units=200, return_sequences=True))
    model.add(Dropout(0.5))  # Regularización para evitar sobreajuste
    model.add(LSTM(units=100, return_sequences=True))
    model.add(Dropout(0.4))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(units=25, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))  # Salida para predicción
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Función para crear ventanas de secuencia a partir de un DataFrame
def crear_ventanas_secuencia(df, window_size, step_size):
    X = []
    y = []

    for i in range(0, len(df) - window_size + 1, step_size):
        ventana = df.iloc[i:i + window_size]
        X.append(ventana['velocity(m/s)'].values.reshape(-1, 1))
        etiqueta_ventana = ventana['label'].max()  # Si hay al menos un '1', la ventana es 1
        y.append(etiqueta_ventana)

    return np.array(X), np.array(y)

# Función para aplicar transformaciones ligeras a las secuencias (agregar ruido)
def aplicar_transformaciones(X, ruido_factor=0.05):
    X_transformado = X + ruido_factor * np.random.normal(loc=0.0, scale=1.0, size=X.shape)
    return X_transformado

# Función para sobremuestrear la clase minoritaria
def sobremuestrear_clase_minoritaria(X, y):
    # Separar clases
    X_mayoritaria = X[y == 0]
    X_minoritaria = X[y == 1]

    # Sobremuestrear la clase minoritaria aplicando duplicación con ligeras transformaciones
    X_minoritaria_duplicada = [aplicar_transformaciones(seq) for seq in X_minoritaria]

    # Reescalar para igualar la cantidad de secuencias de la clase mayoritaria
    X_minoritaria_duplicada = resample(X_minoritaria_duplicada, replace=True, n_samples=len(X_mayoritaria), random_state=42)

    # Concatenar las secuencias originales con las sobremuestreadas
    X_balanceado = np.concatenate([X_mayoritaria, X_minoritaria_duplicada], axis=0)
    y_balanceado = np.array([0] * len(X_mayoritaria) + [1] * len(X_minoritaria_duplicada))

    return X_balanceado, y_balanceado

# Función para procesar archivos .csv en lotes
def procesar_lote_de_eventos(lote_archivos, carpeta_eventos, window_size, step_size):
    X_total = []
    y_total = []

    for archivo in lote_archivos:
        if archivo.endswith('.csv'):
            ruta_archivo = os.path.join(carpeta_eventos, archivo)
            df_evento = pd.read_csv(ruta_archivo)
            X, y = crear_ventanas_secuencia(df_evento, window_size, step_size)
            X_total.append(X)
            y_total.append(y)

    X_total = np.concatenate(X_total, axis=0)
    y_total = np.concatenate(y_total, axis=0)

    return X_total, y_total

# Función para procesar archivos .csv
def procesar_eventos(lista_eventos, carpeta_eventos, window_size, step_size):
    X_total = []
    y_total = []

    for archivo in lista_eventos:
      df_evento = pd.read_csv(os.path.join(carpeta_eventos, archivo))
      X, y = crear_ventanas_secuencia(df_evento, window_size, step_size)
      X_total.append(X)
      y_total.append(y)

    X_total = np.concatenate(X_total, axis=0)
    y_total = np.concatenate(y_total, axis=0)

    return X_total, y_total

# Función para calcular pesos de las clases
def calcular_pesos_clases(y):
    class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
    return dict(enumerate(class_weights))

# Entrenamiento por lotes
def entrenar_por_lotes(carpeta_eventos, model, window_size, step_size, batch_size, num_epochs, save_path):
    archivos_eventos = [f for f in os.listdir(carpeta_eventos) if f.endswith('.csv')]

    for epoch in range(num_epochs):
        print(f"Época {epoch + 1}/{num_epochs}")

        for i in range(0, len(archivos_eventos), batch_size):
            # Obtener un lote de archivos
            lote_archivos = archivos_eventos[i:i + batch_size]

            # Procesar el lote de eventos
            X_total, y_total = procesar_lote_de_eventos(lote_archivos, carpeta_eventos, window_size, step_size)

            # Aplicar sobremuestreo manual si hay desbalance en las clases
            if len(np.unique(y_total)) == 2 and (np.sum(y_total == 1) / len(y_total)) < 0.5:
                X_balanced, y_balanced = sobremuestrear_clase_minoritaria(X_total, y_total)
            else:
                X_balanced, y_balanced = X_total, y_total

            # Calcular pesos de las clases para el conjunto actual
            class_weights = calcular_pesos_clases(y_balanced)

            # Entrenar el modelo en el lote actual
            model.fit(X_balanced, y_balanced, epochs=10, batch_size=128, validation_split=0.3, verbose=1, class_weight=class_weights)

        # Guardar el modelo después de cada época
        model_name = os.path.join(save_path, f'lstm_lunar_model_epoch_{epoch + 1}.keras')
        model.save(model_name)
        print(f"Fin de la época {epoch + 1}")

    # Guardar el modelo final
    model.save(os.path.join(save_path, f'lstm_lunar_model.keras'))

    return model

# Entrenamiento
def entrenar(carpeta_eventos, model, window_size, step_size, num_epochs, save_path):
    archivos_eventos = [f for f in os.listdir(carpeta_eventos) if f.endswith('.csv')]

    # Procesar los eventos
    X_total, y_total = procesar_eventos(archivos_eventos, carpeta_eventos, window_size, step_size)

    # Aplicar sobremuestreo manual si hay desbalance en las clases
    if len(np.unique(y_total)) == 2 and (np.sum(y_total == 1) / len(y_total)) < 0.5:
        X_balanced, y_balanced = sobremuestrear_clase_minoritaria(X_total, y_total)
    else:
        X_balanced, y_balanced = X_total, y_total

    # Calcular pesos de las clases para el conjunto actual
    class_weights = calcular_pesos_clases(y_balanced)

    # Definir el callback para guardar el mejor modelo en base a la métrica de validación
    checkpoint = ModelCheckpoint(filepath='mejor_modelo.keras', monitor='val_loss',
                                 verbose=1, save_best_only=True, save_weights_only=False, mode='min')

    # Entrenar el modelo en el lote actual
    model.fit(X_balanced, y_balanced, epochs=num_epochs, batch_size=32,
              validation_split=0.3, class_weight=class_weights,
              callback = [checkpoint])

    # Guardar el modelo
    model_name = 'lstm_lunar_model.keras'
    model.save(os.path.join(save_path, model_name))

    return model

# Función para plotear métricas
def plot_training_history(history):
    # Graficar la pérdida (loss) durante el entrenamiento y validación
    plt.figure(figsize=(12, 6))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
    plt.plot(history.history['val_loss'], label='Pérdida de validación')
    plt.title('Pérdida durante el entrenamiento y validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida (Loss)')
    plt.legend()

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Precisión de entrenamiento')
    plt.plot(history.history['val_accuracy'], label='Precisión de validación')
    plt.title('Precisión durante el entrenamiento y validación')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión (Accuracy)')
    plt.legend()

    # Mostrar gráficos
    plt.tight_layout()
    plt.show()

# Parámetros
window_size = 100
step_size = 10
batch_size = 5  # Procesar 5 archivos por lote
num_epochs = 10
carpeta_eventos = '' # Same path to the directory where are the csv processed

# Directorio donde se encuetran los modelos
actual_time = datetime.now()
model_path = ''
crear_directorio_si_no_existe(model_path)

# Crear el modelo o cargar uno previamente entrenado
model_name = 'lstm_lunar_model.keras'
if os.path.exists(os.path.join(model_path, model_name)):
    model = load_model(os.path.join(model_path, model_name))
    print(f"Modelo cargado desde {os.path.join(model_path, model_name)}")
else:
    model = crear_modelo_lstm()

# Entrenamiento
history = entrenar_por_lotes(carpeta_eventos, model, window_size, step_size, batch_size, num_epochs, model_path)
#history = entrenar(carpeta_eventos, model, window_size, step_size, num_epochs, model_path)

# Ploteo de graficas
plot_training_history(history)

Época 1/20


# Evaluación del modelo

For the evaluation of the model is necessary to set the following variables:

graphs_path (path to save the graphs plotted)
carpeta_test (path where are the directories of test dataset)
model_path (same path where is alocate the best model)
catalogo_path (path to the output catalog with the detections)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from datetime import datetime, timedelta

# Función para crear ventanas de secuencia
def crear_ventanas_secuencia_prediccion(df, window_size, step_size, columna):
    X = []
    indices = []

    # Crear ventanas deslizantes a partir de la columna especificada
    for i in range(0, len(df) - window_size + 1, step_size):
        ventana = df[columna].iloc[i:i + window_size]
        X.append(ventana.values.reshape(-1, 1))  # Reshape para que sea compatible con la LSTM
        indices.append(i)  # Guardar el índice inicial de cada ventana

    return np.array(X), np.array(indices)

# Función para graficar predicciones y marcar el inicio predicho por el modelo
def graficar_predicciones(df, predicciones, indice_ventana_max, window_size, graphs_path):
    file_name = os.path.basename(archivo)
    save_path = os.path.join(graphs_path, os.path.splitext(file_name)[0])

    # Graficar la señal de velocidad
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df['velocity(m/s)'], label='Señal de Velocidad')

    # Resaltar el inicio predicho (ventana donde se alcanzó el máximo)
    inicio_predicho = indice_ventana_max
    plt.axvline(x=inicio_predicho, color='red', linestyle='--', label='Inicio Predicho')

    plt.title(f'Predicciones para archivo {file_name}')
    plt.xlabel('Índice de Tiempo')
    plt.ylabel('Velocidad (m/s)')
    plt.legend()
    plt.savefig(f'{save_path}.png', dpi=300, bbox_inches='tight')
    #plt.show()

# Parámetros
window_size = 100
step_size = 10
graphs_path = ''
carpeta_test = '' # space_apps_2024_seismic_detection/data/lunar/test/data
model_path = ''
catalogo_path = ''

# Cargar el modelo entrenado
model = load_model(model_path)

# Lista para almacenar las predicciones
predicciones_catalogo = []

# Procesar archivos de test
subcarpetas_test = [f for f in os.listdir(carpeta_test)]
for subcarpeta in subcarpetas_test:
    subcarpeta_path = os.path.join(carpeta_test, subcarpeta)
    archivos_test = [f for f in os.listdir(subcarpeta_path) if f.endswith('.csv')]
    for archivo in archivos_test:
        ruta_archivo = os.path.join(subcarpeta_path, archivo)
        df_test = pd.read_csv(ruta_archivo)

        # Crear ventanas de secuencia para testeo
        X_test, indices = crear_ventanas_secuencia_prediccion(df_test, window_size, step_size, columna='velocity(m/s)')

        # Hacer predicciones
        predicciones = model.predict(X_test)

        # Encontrar la ventana con la predicción más alta
        indice_ventana_max = indices[np.argmax(predicciones)]

        # Calcular el tiempo absoluto y tiempo relativo
        tiempo_abs = df_test['time_abs(%Y-%m-%dT%H:%M:%S.%f)'].iloc[indice_ventana_max]
        tiempo_abs = pd.to_datetime(tiempo_abs).strftime('%Y-%m-%dT%H:%M:%S.%f')

        tiempo_rel = (indice_ventana_max * step_size)  # Tiempo relativo en segundos

        # Guardar la información en la lista del catálogo
        predicciones_catalogo.append([archivo, tiempo_rel])

        # Graficar los resultados
        graficar_predicciones(df_test, predicciones, indice_ventana_max, window_size, graphs_path)

# Crear un DataFrame y guardar en CSV
df_catalogo = pd.DataFrame(predicciones_catalogo, columns=['filename', 'time_rel(sec)'])
df_catalogo.to_csv(catalogo_path, index=False)

print(f"Catálogo de predicciones guardado en {catalogo_path}")